## 1. Init

In [2]:
import os
import os.path as op
import nltk
import re
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
from collections import Counter

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from func import *

## 2. Feature Engineering
### 2.1 Blogs

In [ ]:
#Mine POS patterns
tagList = [
    'NN', 'CC', 'LS', 'PDT', 'POS', 'SYM', 'NNS', 'NNP', 'NNPS', 'FW', 'CD',
    'JJ', 'JJR', 'JJS', 'IN', 'TO', 'DT', 'EX', 'PRP', 'PRP$', 'WDT', 'WP',
    'WP$', 'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG', 'RB', 'RBR', 'RBS',
    'RP', 'WRB', 'UH', '.'
]

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            text = gettext(txtDir + m + '/' + file)
            sentences = nltk.sent_tokenize(text)
            CorpusPOS(sentences)

infile = open('CorpusPOS.txt', 'r')
cPOS = infile.readlines()
infile.close()
(a, b, c, d, e) = calc_probabilities(cPOS)
q1_output(a, b, c, d, e)

Prob = {}
infile = open('probabilities.txt', 'r')
prob_text = infile.readlines()

for sentence in prob_text:
    keyValPair = sentence.split(":")
    Prob[keyValPair[0]] = float(keyValPair[1][:-1])

infile.close()

posFeatures = minePOSPats(cPOS)

In [ ]:
#Feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
labels = []

for gender in [0, 1]:
    if gender == 0:
        txtDir = './blogs-2000/female/'
    else:
        txtDir = './blogs-2000/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in tqdm(os.listdir(txtDir + m), leave=False):
            name = txtDir + m + '/' + file
            text = gettext(name)
            words = nltk.word_tokenize(text)
            tags = nltk.pos_tag(words)
            words_l = wordlemmatize(tags)

            F_feature = F_measure(tags)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            labels.append(gender)

WC_features_l = []
GRF_features_l = []

for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)

F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_blogs-2000.csv', index=False)

In [ ]:
#Get single
WC_features_l = []
for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

GRF_features_l = []
for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

POS_features_l = []
for i in range(len(POS_features[0])):
    n = i
    POS_features_l.append(getsingle(POS_features, n))

In [ ]:
#Get DataFrame
map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

for i in range(len(POS_features[0])):
    key = 'POS_' + str(i + 1)
    value = POS_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)
F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_blogs.csv', index=False)
genderbias = pd.read_csv('blogs_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('blogs_features.csv', index=False)

In [ ]:
#joint
for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
        outdir = './blogs-2000/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'
        outdir = './blogs-2000/male/'

    if not op.exists(outdir):
        os.makedirs(outdir)
    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        handle = ''
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in tqdm(os.listdir(txtDir + m),leave=False):
            name = txtDir + m + '/' + file
            text = gettext(name)
            handle = handle + ' ' + text
        os.mkdir(op.join(outdir, m))
        path = op.join(outdir, m, f'{m}.txt')
        with open(path, 'w') as w:
            w.write(handle)

In [ ]:
#cut
for gender in [0, 1]:
    if gender == 0:
        txtDir = './blogs-2000/female/'
    else:
        txtDir = './blogs-2000/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        print("Processing: ", m)
        filepath = op.join(txtDir, m, f'{m}.txt')
        text = gettext(filepath)
        words = nltk.word_tokenize(text)
        imax = len(words) // 2000
        for i in tqdm(range(imax),leave=False):
            handle = words[i * 2000:(i + 1) * 2000]
            handle = ' '.join(handle)
            savepath = op.join(txtDir, m, f'{m}_{i+1}.txt')
            with open(savepath, 'w') as w:
                w.write(handle)
    

### 2.2 Novels

In [ ]:
#Mine POS patterns
tagList = [
    'NN', 'CC', 'LS', 'PDT', 'POS', 'SYM', 'NNS', 'NNP', 'NNPS', 'FW', 'CD',
    'JJ', 'JJR', 'JJS', 'IN', 'TO', 'DT', 'EX', 'PRP', 'PRP$', 'WDT', 'WP',
    'WP$', 'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG', 'RB', 'RBR', 'RBS',
    'RP', 'WRB', 'UH', '.'
]
namelist = []
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    handle_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(handle_gender))):
        m = handle_gender[i]
        path = op.join(txtDir, m)
        text = gettext(path)
        sentences = nltk.sent_tokenize(text)
        name = op.join(txtDir, 'pos' + m)
        CorpusPOS(sentences, name)
        namelist.append(name)

infile = open('CorpusPOS.txt', 'r')
cPOS = infile.readlines()
infile.close()
(a, b, c, d, e, f, g) = calc_probabilities(cPOS)
q1_output(a, b, c, d, e, f, g)

Prob = {}
infile = open('probabilities.txt', 'r')
prob_text = infile.readlines()

for sentence in prob_text:
    keyValPair = sentence.split(":")
    Prob[keyValPair[0]] = float(keyValPair[1][:-1])

infile.close()

posFeatures = minePOSPats(namelist)

In [ ]:
#Word count and feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
labels = []
lengths = []
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    handle_gender = os.listdir(txtDir)
    print("Files:", len(handle_gender))
    for m in handle_gender:
        name = txtDir + m
        text = gettext(name)
        words = nltk.word_tokenize(text)
        tags = nltk.pos_tag(words)
        words_l = wordlemmatize(tags)

        length = len(words)
        F_feature = F_measure(tags)
        GRF_feature = Gender_Preferential_Features(words_l)
        WC_feature = Word_Classes_Feature(words_l)

        names.append(name)
        lengths.append(length)
        F_features.append(F_feature)
        GRF_features.append(GRF_feature)
        WC_features.append(WC_feature)
        labels.append(gender)

WC_features_l = []
GRF_features_l = []

for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

map1 = {
    'name': names,
    'label': labels,
    'F_feature': F_features,
    'word count': lengths
}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)

F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_5000.csv', index=False)
genderbias = pd.read_csv('5000_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('5000_features.csv', index=False)

In [ ]:
#new feature
swnPos, swnNeg = preforsen()
baseFs = []
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    handle_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(handle_gender))):
        m = handle_gender[i]
        text = op.join(txtDir, m)
        text = gettext(text)
        words = nltk.word_tokenize(text)
        sentences = nltk.sent_tokenize(text)
        basefeature = baseFeatures(text, words, sentences, swnPos, swnNeg)
        baseFs.append(basefeature)

In [ ]:
allofall['word count'].describe()

In [ ]:
#shorten text
#length_list = [200, 400, 600, 800, 1200, 1600, 2000, 27850]
length_list = [12000, 14000, 16000, 18000]

for l in length_list:
    print('\nPresent length:{}'.format(l))
    for gender in [0, 1]:
        if gender == 0:
            txtDir = './5000/female/'
            saveDir = f'./5000-{l}/female/'
        else:
            txtDir = './5000/male/'
            saveDir = f'./5000-{l}/male/'
        if not op.isdir(f'./5000-{l}'):
            os.mkdir(f'./5000-{l}')
        if not op.isdir(saveDir):
            os.mkdir(saveDir)
        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for m in tqdm(handle_gender):
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            if len(words) < (l + 1):
                continue
            words = words[:l]
            text = ''
            for word in words:
                text = text + word + ' '
            with open(op.join(saveDir, m), 'w') as f:
                f.write(text)

In [ ]:
#cut
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
        outdir = './5000-2000/female/'
    else:
        txtDir = './5000/male/'
        outdir = './5000-2000/male/'

    if not op.exists(outdir):
        os.makedirs(outdir)
    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        filepath = op.join(txtDir, m)
        text = gettext(filepath)
        words = nltk.word_tokenize(text)
        imax = len(words) // 2000
        for i in tqdm(range(imax),leave=False):
            handle = words[i * 2000:(i + 1) * 2000]
            handle = ' '.join(handle)
            savepath = op.join(outdir, f'{m[:-4]}_{i+1}.txt')
            with open(savepath, 'w') as w:
                w.write(handle)

In [ ]:
#feature extract
length_list = [2000]
for l in length_list:
    names = []
    F_features = []
    GRF_features = []
    WC_features = []
    labels = []
    print('\nPresent length:{}'.format(l))
    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'./5000-{l}/female/'
        else:
            txtDir = f'./5000-{l}/male/'

        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for m in tqdm(handle_gender):
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            tags = nltk.pos_tag(words)
            words_l = wordlemmatize(tags)

            F_feature = F_measure(tags)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            labels.append(gender)

    WC_features_l = []
    GRF_features_l = []

    for i in range(len(WC_features[0])):
        n = i
        WC_features_l.append(getsingle(WC_features, n))

    for i in range(len(GRF_features[0])):
        n = i
        GRF_features_l.append(getsingle(GRF_features, n))

    map1 = {'name': names, 'label': labels, 'F_feature': F_features}

    for i in range(len(WC_features[0])):
        key = 'WC_' + str(i + 1)
        value = WC_features_l[i]
        map1[key] = value

    for i in range(len(GRF_features[0])):
        key = 'GRF_' + str(i + 1)
        value = GRF_features_l[i]
        map1[key] = value

    allofall = pd.DataFrame(map1)

    F_features_u = np.array(F_features)
    F_features_u = (F_features_u -
                    np.mean(F_features_u)) / np.std(F_features_u)
    allofall['F_feature'] = F_features_u

    allofall.to_csv(f'allofall_5000-{l}.csv', index=False)
    #genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    #allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias['word ratio']
    #allofall.to_csv(f'5000-{l}_features.csv',index = False)

## 3. Train

In [12]:
def train(clf, data, target):
    scores_clf_cv = cross_val_score(clf,
                                    data.values,
                                    target.values.reshape(-1, ),
                                    cv=5)
    print(scores_clf_cv)
    print("Accuracy: %0.6f (+/- %0.3f)" %
          (scores_clf_cv.mean(), scores_clf_cv.std() * 2))
    return scores_clf_cv.mean(), clf.feature_importances_

In [ ]:
#train
allf = []
allf_i = []
gb = []
gb_i = []
without = []
without_i = []
featurelist = []
length_list = [12000, 14000, 16000, 18000]
list1 = ['occu', 'emo', 'verb']

for l in length_list:
    allofall = pd.read_csv(f'allofall_5000-{l}.csv')
    genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
        'word ratio']

    for j in range(3):
        genderbias = pd.read_csv(f'5000-{l}_genderbias-{list1[j]}.csv')
        allofall[f'bias-{list1[j]}'], allofall[
            f'word ratio{list1[j]}'] = genderbias['bias'], genderbias[
                'word ratio']

    allofall.to_csv(f'5000-{l}_features.csv', index=False)
    data, target = allofall.drop(
        columns=['name', 'label']).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))
    data1 = data.iloc[:, :-6]
    data2 = data.iloc[:, :-8]
    data3 = data.iloc[:, -8:-6]
    rnd_clf = RandomForestClassifier(n_estimators=800,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)

    print('\n Length:{}'.format(l))
    a, b = train(rnd_clf, data1, target)
    allf.append(a)
    allf_i.append([b])
    a, b = train(rnd_clf, data2, target)
    without.append(a)
    without_i.append([b])
    a, b = train(rnd_clf, data3, target)
    gb.append(a)
    gb_i.append([b])

In [ ]:
#length vs accuracy
df = pd.read_csv('length_v_acc_and_im.csv')
df = df.set_index('length')
allofall = pd.read_csv('5000-10000_features.csv')

allf_ii = [0] * len(length_list)
without_ii = [0] * len(length_list)
gb_ii = [0] * len(length_list)
name = allofall.columns[2:]
for i in range(len(length_list)):
    allf_ii[i] = sorted(list(zip(name, allf_i[i][0])),
                        key=lambda x: x[1],
                        reverse=True)
    without_ii[i] = sorted(list(zip(name[:-2], without_i[i][0])),
                           key=lambda x: x[1],
                           reverse=True)
    gb_ii[i] = sorted(list(zip(name[-2:], gb_i[i][0])),
                      key=lambda x: x[1],
                      reverse=True)

allf_iii = [0] * len(length_list)
without_iii = [0] * len(length_list)
name = ['F', 'WC', 'GRF', 'bias', 'word ratio']
for i in range(len(length_list)):
    temp = allf_i[i]
    handle = [0] * 5
    handle[0] = temp[0][0]
    handle[1] = sum(temp[0][1:24])
    handle[2] = sum(temp[0][24:34])
    handle[3:] = temp[0][-2:]
    handle = sorted(zip(name, handle), key=lambda x: x[1], reverse=True)
    allf_iii[i] = handle
    temp = without_i[i]
    handle = [0] * 3
    handle[0] = temp[0][0]
    handle[1] = sum(temp[0][1:24])
    handle[2] = sum(temp[0][24:34])
    handle = sorted(zip(name[:-2], handle), key=lambda x: x[1], reverse=True)
    without_iii[i] = handle

start = 8
df1 = df.iloc[:8, :]
df2 = df.iloc[:len(length_list),:]
df2.index = length_list
df3 = df.iloc[8:, :]
df = pd.concat([df1, df2, df3])
for i in range(len(length_list)):
    df.iloc[start + i, 0] = allf[i]
    df.iloc[start + i, 3] = without[i]
    df.iloc[start + i, 6] = gb[i]
    df.iloc[start + i, 1] = [allf_ii[i][:5]]
    df.iloc[start + i, 4] = [without_ii[i][:5]]
    df.iloc[start + i, 7] = [gb_ii[i]]
    df.iloc[start + i, 2] = [allf_iii[i]]
    df.iloc[start + i, 5] = [without_iii[i]]
#df.to_csv('top5_sum.csv')

## 4. Others

In [ ]:
#10wincv
from sklearn.model_selection import cross_val_score

for i in range(1, 11):
    fname = str(10 + i) + '_win_5000_genderbias.csv'
    genderbias = pd.read_csv(fname)
    allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
        'word ratio']
    data, target = allofall.drop(columns=['name', 'label', 'word count']
                                 ).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))

    rnd_clf = RandomForestClassifier(n_estimators=1000,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Windows:%.2f'%(i))
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

In [ ]:
#train
allf = []
allf_i = []
bias1 = []
bias1_i = []
bias2 = []
bias2_i = []
bias3 = []
bias3_i = []
list1 = ['occu', 'emo', 'verb']

for l in length_list:
    allofall = pd.read_csv(f'allofall_5000-{l}.csv')
    for j in range(3):
        genderbias = pd.read_csv(f'5000-{l}_genderbias-{list1[j]}.csv')
        allofall[f'bias-{list1[j]}'], allofall[
            f'word ratio{list1[j]}'] = genderbias['bias'], genderbias[
                'word ratio']
    allofall.to_csv(f'5000-{l}_features.csv', index=False)
    data, target = allofall.drop(
        columns=['name', 'label']).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))
    data1 = data.iloc[:, :-6]
    data2 = data.iloc[:, :-8]
    data3 = data.iloc[:, -8:-6]
    rnd_clf = RandomForestClassifier(n_estimators=800,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)

    print('\n Length:{}'.format(l))
    a, b = train(rnd_clf, data1, target)
    allf.append(a)
    allf_i.append([b])
    a, b = train(rnd_clf, data2, target)
    bias1.append(scores_rnd_clf_cv.mean())
    bias1_i.append([rnd_clf.feature_importances_])
    a, b = train(rnd_clf, data3, target)
    bias2.append(scores_rnd_clf_cv.mean())
    bias2_i.append([rnd_clf.feature_importances_])

    bias3.append(scores_rnd_clf_cv.mean())
    bias3_i.append([rnd_clf.feature_importances_])

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor, RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_leaf_nodes=32),
                             n_estimators=200,
                             algorithm="SAMME.R",
                             learning_rate=1,
                             random_state=42)
ada_clf.fit(train_X, train_y)
ada_clf.score(test_y, y_pred)

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=1000,
                                 max_leaf_nodes=64,
                                 n_jobs=-1)
gbrt = GradientBoostingRegressor(max_leaf_nodes=64,
                                 n_estimators=10,
                                 learning_rate=1.0)

In [ ]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(random_state=42)
mlp_clf = MLPClassifier(random_state=42)

estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(train_X, train_y)

[estimator.score(test_X, test_y) for estimator in estimators]

In [ ]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]
voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)
voting_clf.score(X_val, y_val)

In [ ]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

In [ ]:
voting_clf.set_params(extra_trees_clf=None)

In [ ]:
del voting_clf.estimators_[1]
voting_clf.score(X_val, y_val)

In [ ]:
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

In [ ]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [ ]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)
rnd_forest_blender.score(X_val_predictions,y_val)

In [ ]:
rnd_forest_blender.oob_score_

In [8]:
f = ['bias', 'word ratio', 'base_15', 'GRF_8', 'WC_2', 'GRF_1']
allofall = pd.read_csv('newnewnew.csv')
data, target = allofall.drop(
    columns=['name', 'label', 'word count']).iloc[:], allofall['label']

In [10]:
data1 = data[f]
clf = RandomForestClassifier(n_estimators=800, max_leaf_nodes=64, n_jobs=-1)

In [13]:
a,b = train(clf,data1,target)

[0.728 0.762 0.78  0.771 0.76 ]
Accuracy: 0.760200 (+/- 0.035)


In [14]:
a,b = train(clf,data,target)

[0.751 0.773 0.8   0.803 0.78 ]
Accuracy: 0.781400 (+/- 0.038)
